# CSV Note from Jacob Abba

In [1]:
'''
Note from Jacob Abba defining data format for CSV Patient ECG Files:


The csv is a sample of the format that I’ll be providing. 

Each CSV file will represent a single admission in a bed, 
and its filename is an encrypted version of that admission’s ID in our database.
 
Each row in the CSV represents a PVC-type alarm from that admission. 
The first column is an encrypted version of the alarm’s ID in our database. 
The second column is the time that the alarm occurred (in number of seconds since the start of the admission). 
The third column is the strip data for that alarm, which has been deflated and base64 encoded. 
Note that the rows in these files aren’t sorted in any particular order.
 
To get the strip data, simply use a library in your language of choice to read the base64 string to a buffer, 
deflate the buffer, and then encode the buffer into a string. In nodejs it looks something like this:
 
let zlib = require(‘zlib’);
 
let s = strip; //load strip
let buf = Buffer.from(s, 'base64'); //read strip into buffer
let json = zlib.unzipSync(buf).toString(); //deflate buffer and convert to a string
 
Once decoded, the strip is an array in JSON format and looks something like this (without the pretty print):
 
[
    {"Label":"I","ID":"7","Text":"-263,-211,-146,-108,-68,-35,3,…”},
    {"Label":"II","ID":"8","Text":"-312,-272,-220,-177,-137,-111,…”},
    …
]
 
Pretty self-descriptive; each object within the array represents a channel where 
the “Label” field contains the label, 
the “Text” field contains the waveform (with each sample separated by a comma). 
You can probably ignore the “ID” field, it’s just used internally by bedmaster.
 
A few notes on these strips:
1)      In our meeting Xiao and Ran said that they contain data 5 seconds before and 5 seconds after an alarm. 
As far as I can tell this is not correct; they seem to only contain data 10 seconds before the alarm. 
For your purposes this shouldn’t matter, but it’s good to know.

2)      Sampling frequency of the channels is always 240hz.

3)      Some files may contain different numbers of channels. 
Originally we were planning on only using files that had the 
8 channels (‘I’, ‘II’, ‘III’, ‘V’, ‘AVR’, ‘AVL’, ‘AVF’, and ‘SPO2’), 
and discarding other channels. But Ran and Xiao would be able to guide you better on how to handle this.

 
For the rules to automatically sort pvc alarms into true- and false-positive categories, 
you’ll also need timing of artifact alarms for each admission. 
I’ll work on a script to generate these as well, but it should be simpler since they don’t need to include strips. 
My plan is to just have text files for each admission (same name as the csv) 
with each line representing the relative timing of that alarm.
 
You can see Xiao’s earlier email with documentation on the adibin file format.
 
Hopefully this info is helpful to you, 
if anything is unclear or you need guidance on data conversion/reading the csv then 
feel free to reach out via text or email!
 
Jacob
'''

print('')

# ADIBinaryFormat.h
### Shows structure of File Header and Channel Titles 

In [2]:
'''
Code in ADIBinaryFormat.h that shows the structure of the FileHeader and Channel Titles

/***************************************************************************
 * Translate Binary for LabChart for Windows
 *
 * ADIBinaryFormat.h
 *
 * Copyright (c) 2001-2009 ADInstruments Ltd.
 *
 * Translate Binary is a for Windows extension that enables data to be
 * moved to and from LabChart, in a simple binary format.
 *
 * A LCfW binary file has the following structure:
 *
 *  - A 68 byte file header (CFWBINARY structure) containing basic information
 *   about the data such as the sampling period, number of channels, trigger
 *   time, data format.
 *
 *  - For each channel, a 96 byte channel header (CFWBCHANNEL structure)
 *   containing information about the channel.
 *
 *  - The interleaved channel data. Data can be either double precision
 *   floating point, single precision floating point or 16 bit integer, as
 *   specified by the DataFormat parameter of the file header.
 *
 *
 * The CFWBINARY and CFWBCHANNEL structures are defined below, along with a
 * simple program which creates a two channel CfW binary file.
 *
 * Note that the following types must have the indicated size and that the
 * program needs to be run on a little endian machine (e.g. x86):
 *
 * sizeof(char)   = 1 byte
 * sizeof(short)  = 2 bytes
 * sizeof(long)   = 4 bytes
 * sizeof(double) = 8 bytes, i.e. 64 bit IEEE floating point
 *
 ******************************************************************************/
#ifndef _ADIBinFormat
#define _ADIBinFormat

#define CHANNEL_TITLE_LEN  32
#define UNITS_LEN          32
#define CFWB_VERSION       1

enum 
   {
   kBinFmtDouble  =  1,
   kBinFmtFloat,
   kBinFmtInt16,
   };

// The file and header structures must be packed on 1-byte boundaries.
// On Visual C++ the following pragma enforces this.
#pragma pack(1)

// Each LabChart for Windows binary file starts with the following structure:
struct CFWBINARY
   {
   char     magic[4];            // always "CFWB"
   long     Version;             // = CFWB_VERSION
   double   secsPerTick;         // sampling interval in seconds

   // Trigger Date and time information
   long    Year;                // 4 digit year
   long    Month;               // months 1 - 12
   long    Day;                 // days 1 - 31
   long    Hour;                // hours 0 - 23
   long    Minute;              // minutes 0 - 59
   double   Second;              // seconds
   double   trigger;             // Amount of pretrigger data in seconds.

   long     NChannels;           // Number of channels
   long     SamplesPerChannel;   // Number of sample points per channel
   
   // The TimeChannel flag indicates that the sample time of each sample is
   // interleaved as the first column of data. This is only valid for the floating
   // point data formats. For all releases of TranslateBinary up to and including
   // v1.3, sample time data can be included but it is not used.
   long     TimeChannel;         // 1 = time included as first channel, 0 = not included
   long     DataFormat;          // 1 = double , 2 = float, 3 = 16-bit integer
   };

// Then one of these for each of the 'NChannels':
struct CFWBCHANNEL
   {
   char     Title[CHANNEL_TITLE_LEN];  // Channel title string
   char     Units[UNITS_LEN];          // Channel units string

   // scale and offset are used to convert 16-bit samples into user units,
   // where  data = scale * (sample + offset)
   double   scale;                     // scale (= 1.0 for floating point data)
   double   offset;                    // offset (= 0.0 for floating point data)

   // The maximum and minimum values of the data
   // (not used in TranslateBinary up to and including v1.3)
   double   RangeHigh;
   double   RangeLow;
   };

// Back to default data structure packing
#pragma pack()

#endif   // sentinelRangeLow;
   };

// Back to default data structure packing
#pragma pack()

#endif   // sentinel

'''
print('')